## Simulated annealing

In [ ]:
pip install scikit-optimize

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st
import timeit
from joblib import Parallel, delayed
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

In [1]:
start = timeit.default_timer()

def calculate_safety_stock(service_level, std_dev_daily_sales, lead_time):
    z = st.norm.ppf(service_level)  
    safety_stock = z * std_dev_daily_sales * np.sqrt(lead_time)
    return safety_stock

summary = {
    'Purchase Cost': [12, 7, 6, 37],
    'Lead Time': [9, 6, 15, 12],
    'Size': [0.57, 0.05, 0.53, 1.05],
    'Selling Price': [16.10, 8.60, 10.20, 68],
    'Starting Stock': [2750, 22500, 5200, 1400],
    'Ch': [0.2 * 12, 0.2 * 7, 0.2 * 6, 0.2 * 37],
    'Co': [1000, 1200, 1000, 1200],
    'Probability': [0.76, 1.00, 0.70, 0.23],
    'Mean Demand (Lead Time)': [103.50, 648.55, 201.68, 150.06],
    'Std. Dev. of Demand (Lead Time)': [37.32, 26.45, 31.08, 3.21],
    'Expected Demand (Lead Time)': [705, 3891, 2266, 785]
}

class Product:
    def __init__(self, i):
        self.mean = np.mean([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.sd = np.std([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.i = i
        self.unit_cost = summary['Purchase Cost'][i - 1]
        self.lead_time = summary['Lead Time'][i - 1]
        self.size = summary['Size'][i - 1]
        self.selling_price = summary['Selling Price'][i - 1]
        self.holding_cost = summary['Ch'][i - 1]
        self.ordering_cost = summary['Co'][i - 1]
        self.probability = summary['Probability'][i - 1]
        self.starting_stock = summary['Starting Stock'][i - 1]
        self.mean_demand_lead_time = summary['Mean Demand (Lead Time)'][i - 1]
        self.std_dev_demand_lead_time = summary['Std. Dev. of Demand (Lead Time)'][i - 1]
        self.expected_demand_lead_time = summary['Expected Demand (Lead Time)'][i - 1]

def daily_demand(mean, sd, probability):
    if np.random.uniform(0, 1) > probability:
        return 0
    else:
        return np.exp(np.random.normal(mean, sd))

def MCS(Q, product, review_period=30, service_level=0.95):
    inventory = product.starting_stock
    mean = product.mean
    sd = product.sd
    lead_time = product.lead_time
    probability = product.probability
    demand_lead = product.expected_demand_lead_time

    """
    - Importance sampling parameters
    - Define a new distribution that emphasizes extreme demand values
    - For simplicity, we used a mixture of two normal distributions
    - One centered around the mean and another centered around a high demand value
    """
    high_demand_value = 2 * mean
    weights = [0.8, 0.2]  
    means = [mean, high_demand_value]
    sds = [sd, sd]  

    daily_sales = [np.random.normal(means[i], sds[i]) for i in np.random.choice(len(weights), size=365, p=weights)]
    max_daily_sales = np.max(daily_sales)
    avg_daily_sales = np.mean(daily_sales)
    std_dev_daily_sales = np.std(daily_sales)

    safety_stock = calculate_safety_stock(service_level, std_dev_daily_sales, lead_time)

    q = 0
    stock_out = 0
    counter = 0
    order_placed = False
    data = {'inv_level': [], 'daily_demand': [], 'units_sold': [], 'units_lost': [], 'orders': [], 'reorder_quantities': []}

    for day in range(1, 365):
        day_demand = [np.random.normal(means[i], sds[i]) for i in np.random.choice(len(weights), size=365, p=weights)]
        if day_demand != 0:
            data['daily_demand'].append(day_demand)

        if day % review_period == 0:
            q = max(0, safety_stock + demand_lead - inventory)
            q = max(q, int(0.2 * mean))
            order_placed = True
            data['orders'].append(q)
            data['reorder_quantities'].append(q)

        if order_placed:
            counter += 1

        if counter == lead_time:
            inventory += q
            order_placed = False
            counter = 0

        for demand in day_demand:
            if inventory - demand >= 0:
                data['units_sold'].append(demand)
                inventory -= demand
            else:
                data['units_sold'].append(inventory)
                data['units_lost'].append(demand - inventory)
                inventory = 0
                stock_out += 1

            data['inv_level'].append(inventory)

    return data, safety_stock

def profit_calculation(data, product):
    unit_cost = product.unit_cost
    selling_price = product.selling_price
    holding_cost = product.holding_cost
    order_cost = product.ordering_cost
    size = product.size
    days = 365

    revenue = sum(data['units_sold']) * selling_price
    Co = len(data['orders']) * order_cost
    Ch = sum(data['inv_level']) * holding_cost * size / days
    cost = sum(data['orders']) * unit_cost

    profit = revenue - cost - Co - Ch

    return profit

# Simulated Annealing functions

def acceptance_probability(current_profit, new_profit, temperature):
    if new_profit > current_profit:
        return 1.0
    return np.exp((new_profit - current_profit) / temperature)

def get_neighbor_solution(Q):
    return max(1, int(Q + np.random.normal(0, 1000)))

def simulated_annealing(product, initial_Q, max_iterations, initial_temperature, cooling_rate):
    current_Q = initial_Q
    current_profit = np.mean([profit_calculation(MCS(current_Q, product)[0], product) for _ in range(10)])  # Average profit from 10 simulations
    best_Q = current_Q
    best_profit = current_profit
    temperature = initial_temperature

    for iteration in range(max_iterations):
        neighbor_Q = get_neighbor_solution(current_Q)
        new_profit = np.mean([profit_calculation(MCS(neighbor_Q, product)[0], product) for _ in range(10)])  # Average profit from 10 simulations

        if acceptance_probability(current_profit, new_profit, temperature) > np.random.random():
            current_Q = neighbor_Q
            current_profit = new_profit

        if current_profit > best_profit:
            best_Q = current_Q
            best_profit = current_profit

        temperature *= cooling_rate

    return best_Q, best_profit

products = [Product(i) for i in range(1, 5)]
results = {}

max_iterations = 100
initial_temperature = 100.0
cooling_rate = 0.95

for product in products:
    best_Q, best_profit = simulated_annealing(product, initial_Q=10000, max_iterations=max_iterations, initial_temperature=initial_temperature, cooling_rate=cooling_rate)

    # best solution
    best_data, best_safety_stock = MCS(best_Q, product)
    profit_list = [profit_calculation(MCS(best_Q, product)[0], product) for _ in range(100)]
    lost_orders_list = [len(MCS(best_Q, product)[0]['units_lost']) / 365 for _ in range(100)]
    reorder_quantities = [best_Q for _ in range(100)]
    safety_stocks = [best_safety_stock for _ in range(100)]

    results[f'Pr{product.i}'] = {
        'profits': profit_list,
        'mean_profit': np.mean(profit_list),
        'std_dev_profit': np.std(profit_list),
        'lost_order_proportion': np.mean(lost_orders_list),
        'reorder_quantity': np.mean(reorder_quantities),
        'average_safety_stock': np.mean(safety_stocks)
    }

results_df = pd.DataFrame.from_dict(results, orient='index')
print(results_df)

stop = timeit.default_timer()
print('Time: ', stop - start)

                                               profits    mean_profit  \
Pr1  [52904.97278417384, 52884.29418409236, 52942.8...   52903.499406   
Pr2  [213962.27501367213, 213991.1590934616, 213949...  213970.726439   
Pr3  [125041.90271581626, 124980.6143209584, 125078...  124998.774419   
Pr4  [312239.8457464758, 312517.0064699763, 311965....  312191.498411   

     std_dev_profit  lost_order_proportion  reorder_quantity  \
Pr1       38.269391             360.309507           10115.0   
Pr2       41.680713             345.923123            5233.0   
Pr3       57.092554             355.051699           13900.0   
Pr4      231.716851             360.510904           12154.0   

     average_safety_stock  
Pr1             12.152267  
Pr2             13.183490  
Pr3             20.079990  
Pr4             14.953049  
Time:  1162.8135186


In [2]:
results_df

,profits,mean_profit,std_dev_profit,lost_order_proportion,reorder_quantity,average_safety_stock
Pr1,"[52904.97278417384, 52884.29418409236, 52942.8...",52903.499406,38.269391,360.309507,10115.0,12.152267
Pr2,"[213962.27501367213, 213991.1590934616, 213949...",213970.726439,41.680713,345.923123,5233.0,13.183490
Pr3,"[125041.90271581626, 124980.6143209584, 125078...",124998.774419,57.092554,355.051699,13900.0,20.079990
Pr4,"[312239.8457464758, 312517.0064699763, 311965....",312191.498411,231.716851,360.510904,12154.0,14.953049


## BayesianOptimization

In [4]:
start = timeit.default_timer()

def calculate_safety_stock(service_level, std_dev_daily_sales, lead_time):
    z = st.norm.ppf(service_level)  
    safety_stock = z * std_dev_daily_sales * np.sqrt(lead_time)
    return safety_stock

summary = {
    'Purchase Cost': [12, 7, 6, 37],
    'Lead Time': [9, 6, 15, 12],
    'Size': [0.57, 0.05, 0.53, 1.05],
    'Selling Price': [16.10, 8.60, 10.20, 68],
    'Starting Stock': [2750, 22500, 5200, 1400],
    'Ch': [0.2 * 12, 0.2 * 7, 0.2 * 6, 0.2 * 37],
    'Co': [1000, 1200, 1000, 1200],
    'Probability': [0.76, 1.00, 0.70, 0.23],
    'Mean Demand (Lead Time)': [103.50, 648.55, 201.68, 150.06],
    'Std. Dev. of Demand (Lead Time)': [37.32, 26.45, 31.08, 3.21],
    'Expected Demand (Lead Time)': [705, 3891, 2266, 785]
}

class Product:
    def __init__(self, i):
        self.mean = np.mean([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.sd = np.std([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.i = i
        self.unit_cost = summary['Purchase Cost'][i - 1]
        self.lead_time = summary['Lead Time'][i - 1]
        self.size = summary['Size'][i - 1]
        self.selling_price = summary['Selling Price'][i - 1]
        self.holding_cost = summary['Ch'][i - 1]
        self.ordering_cost = summary['Co'][i - 1]
        self.probability = summary['Probability'][i - 1]
        self.starting_stock = summary['Starting Stock'][i - 1]
        self.mean_demand_lead_time = summary['Mean Demand (Lead Time)'][i - 1]
        self.std_dev_demand_lead_time = summary['Std. Dev. of Demand (Lead Time)'][i - 1]
        self.expected_demand_lead_time = summary['Expected Demand (Lead Time)'][i - 1]

def daily_demand(mean, sd, probability):
    if np.random.uniform(0, 1) > probability:
        return 0
    else:
        return np.exp(np.random.normal(mean, sd))

def MCS(Q, product, review_period=30, service_level=0.95):
    inventory = product.starting_stock
    mean = product.mean
    sd = product.sd
    lead_time = product.lead_time
    probability = product.probability
    demand_lead = product.expected_demand_lead_time

    """
    - Importance sampling parameters
    - Define a new distribution that emphasizes extreme demand values
    - For simplicity, we used a mixture of two normal distributions
    - One centered around the mean and another centered around a high demand value
    """
    high_demand_value = 2 * mean
    weights = [0.8, 0.2]  
    means = [mean, high_demand_value]
    sds = [sd, sd]  

    daily_sales = [np.random.normal(means[i], sds[i]) for i in np.random.choice(len(weights), size=365, p=weights)]
    max_daily_sales = np.max(daily_sales)
    avg_daily_sales = np.mean(daily_sales)
    std_dev_daily_sales = np.std(daily_sales)

    safety_stock = calculate_safety_stock(service_level, std_dev_daily_sales, lead_time)

    q = 0
    stock_out = 0
    counter = 0
    order_placed = False
    data = {'inv_level': [], 'daily_demand': [], 'units_sold': [], 'units_lost': [], 'orders': [], 'reorder_quantities': []}

    for day in range(1, 365):
        day_demand = [np.random.normal(means[i], sds[i]) for i in np.random.choice(len(weights), size=365, p=weights)]
        if day_demand != 0:
            data['daily_demand'].append(day_demand)

        if day % review_period == 0:
            q = max(0, safety_stock + demand_lead - inventory)
            q = max(q, int(0.2 * mean))
            order_placed = True
            data['orders'].append(q)
            data['reorder_quantities'].append(q)

        if order_placed:
            counter += 1

        if counter == lead_time:
            inventory += q
            order_placed = False
            counter = 0

        for demand in day_demand:
            if inventory - demand >= 0:
                data['units_sold'].append(demand)
                inventory -= demand
            else:
                data['units_sold'].append(inventory)
                data['units_lost'].append(demand - inventory)
                inventory = 0
                stock_out += 1

            data['inv_level'].append(inventory)

    return data, safety_stock

def profit_calculation(data, product):
    unit_cost = product.unit_cost
    selling_price = product.selling_price
    holding_cost = product.holding_cost
    order_cost = product.ordering_cost
    size = product.size
    days = 365

    revenue = sum(data['units_sold']) * selling_price
    Co = len(data['orders']) * order_cost
    Ch = sum(data['inv_level']) * holding_cost * size / days
    cost = sum(data['orders']) * unit_cost

    profit = revenue - cost - Co - Ch

    return profit

# Bayesian Optimization 
def bayesian_optimization(product, initial_Q, n_calls):
    dimensions = [Real(100, 10000, name='Q')]

    @use_named_args(dimensions=dimensions)
    def objective(Q):
        Q = int(Q)
        profit = np.mean([profit_calculation(MCS(Q, product)[0], product) for _ in range(10)])  # Average profit from 10 simulations
        return -profit  # Negative profit because we minimize in gp_minimize

    res = gp_minimize(objective, dimensions=dimensions, n_calls=n_calls, random_state=0)
    best_Q = int(res.x[0])
    best_profit = -res.fun

    return best_Q, best_profit

products = [Product(i) for i in range(1, 5)]
results = {}

n_calls = 100  # Number of calls to the objective function

for product in products:
    best_Q, best_profit = bayesian_optimization(product, initial_Q=10000, n_calls=n_calls)

    # best solution
    best_data, best_safety_stock = MCS(best_Q, product)
    profit_list = [profit_calculation(MCS(best_Q, product)[0], product) for _ in range(100)]
    lost_orders_list = [len(MCS(best_Q, product)[0]['units_lost']) / 365 for _ in range(100)]
    reorder_quantities = [best_Q for _ in range(100)]
    safety_stocks = [best_safety_stock for _ in range(100)]

    results[f'Pr{product.i}'] = {
        'profits': profit_list,
        'mean_profit': np.mean(profit_list),
        'std_dev_profit': np.std(profit_list),
        'lost_order_proportion': np.mean(lost_orders_list),
        'reorder_quantity': np.mean(reorder_quantities),
        'average_safety_stock': np.mean(safety_stocks)
    }


results_df = pd.DataFrame.from_dict(results, orient='index')
print(results_df)

stop = timeit.default_timer()
print('Time: ', stop - start)

C:\Users\sarit.maitra\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\sarit.maitra\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\sarit.maitra\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\sarit.maitra\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Users\sarit.maitra\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective h

                                               profits    mean_profit  \
Pr1  [52906.07232484695, 52910.09902079415, 52885.7...   52904.835498   
Pr2  [213934.7202693551, 214035.15955315033, 213974...  213975.253737   
Pr3  [124955.20954567159, 124958.46504884707, 12499...  124998.185328   
Pr4  [312060.2677582541, 312152.66665353026, 311688...  312133.164927   

     std_dev_profit  lost_order_proportion  reorder_quantity  \
Pr1       38.413531             360.309397            3109.0   
Pr2       41.030428             345.916767            4812.0   
Pr3       63.375288             355.049699            6269.0   
Pr4      199.504568             360.518082            4828.0   

     average_safety_stock  
Pr1             12.740902  
Pr2             12.893567  
Pr3             19.584926  
Pr4             14.442519  
Time:  1411.8059726


In [5]:
results_df

,profits,mean_profit,std_dev_profit,lost_order_proportion,reorder_quantity,average_safety_stock
Pr1,"[52906.07232484695, 52910.09902079415, 52885.7...",52904.835498,38.413531,360.309397,3109.0,12.740902
Pr2,"[213934.7202693551, 214035.15955315033, 213974...",213975.253737,41.030428,345.916767,4812.0,12.893567
Pr3,"[124955.20954567159, 124958.46504884707, 12499...",124998.185328,63.375288,355.049699,6269.0,19.584926
Pr4,"[312060.2677582541, 312152.66665353026, 311688...",312133.164927,199.504568,360.518082,4828.0,14.442519
